# M1. Actividad
### Carlos Remes Inguanzo A01366574
#### 11 de Agosto de 2021

### **Introducción**

En este trabajo se realizará una simulación de un mapa con celdas sucias, habrá un número de aspiradoras que se encargarán de limpiar el lugar avanzando una casilla de alrededor por paso.

### **Setup**

In [1]:
!pip3 install mesa
# La clase `Model` se hace cargo de los atributos a nivel del modelo, maneja los agentes. 
# Cada modelo puede contener múltiples agentes y todos ellos son instancias de la clase `Agent`.
from mesa import Agent, Model 

# Debido a que necesitamos un solo agente por celda elegimos `SingleGrid` que fuerza un solo objeto por celda.
from mesa.space import MultiGrid
from mesa.space import SingleGrid

# Con `SimultaneousActivation` hacemos que todos los agentes se activen de manera simultanea.
from mesa.time import SimultaneousActivation

# Vamos a hacer uso de `DataCollector` para obtener el grid completo cada paso (o generación) y lo usaremos para graficarlo.
from mesa.datacollection import DataCollector

from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer

from mesa.space import ContinuousSpace


# mathplotlib lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Definimos los siguientes paquetes para manejar valores númericos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


### **Variables**

In [2]:
# Limite de movimientos de aspiradora
lim_steps = 100

# Tamaño del mapa
map_width = 10
map_height = 10

# porcentaje (0 - 100) de suciedad en el mapa
# PRECAUCION: Hay un bug que si hay demasiada suciedad se hace invisible (error de Mesa)
dirty_percentage = 30

# Numero de aspiradoras en el mapa
num_vacuums = 6

### **Clases**

In [3]:
def get_grid(model):

    grid = np.zeros((model.grid.width, model.grid.height))
    for cell in model.grid.coord_iter():
        cell_content, x, y = cell
        for content in cell_content:
            if isinstance(content,Vacuum):
                grid[x][y] = 2
            else:
                grid[x][y] = content.live
    return grid

class Dirt(Agent):

    def __init__(self, unique_id, model):

        super().__init__(unique_id, model)
        if np.random.choice(100) <= dirty_percentage:
            self.live = 1
        else:
            self.live = 0
        

class Vacuum(Agent):

    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)

    def move(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos,
            moore=True,
            include_center=False)
                
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self,new_position)
        self.step

    def clean(self):
        this_cell = self.model.grid.get_cell_list_contents([self.pos])
        if this_cell[0].live:
            this_cell[0].live = 0
            return True
        else:
            return False

    def step(self):
        if not self.clean():
            self.move()


class GameModel(Model):
    '''
    Define el modelo del juego de la vida.
    '''
    def __init__(self, width, height):
        self.num_agents = width * height
        self.grid = MultiGrid(width, height, False)
        self.schedule = SimultaneousActivation(self)
        
        for (content, x, y) in self.grid.coord_iter():
            dirt = Dirt((x, y), self)
            self.grid.place_agent(dirt, (x, y))
            self.schedule.add(dirt)
        
        
        for i in range(num_vacuums):
            vacuum = Vacuum(i,self)
            self.grid.place_agent(vacuum, (1, 1))
            self.schedule.add(vacuum)

            
        # Aquí definimos con colector para obtener el grid completo.
        self.datacollector = DataCollector(
            model_reporters={"Grid": get_grid},
            agent_reporters={'Move': lambda a: getattr(a, 'move', None)}
            )

    def count_dirty(self):
        count = 0
        for cell in self.grid.coord_iter():
            cell_content, x, y = cell
            if cell_content[0].live:
                count+=1
        return count

            
    def step(self):
        '''
        En cada paso el colector tomará la información que se definió y almacenará el grid para luego graficarlo.
        '''
        self.datacollector.collect(self)
        self.schedule.step()
    

### **Correr**

In [4]:
start_time = time.time()
model = GameModel(map_width, map_height)
steps_taken = 0
while(model.count_dirty() > 0 and steps_taken < lim_steps):
    model.step()
    steps_taken+=1

# Imprimimos estadísticas.
print('Tiempo de ejecución:', str(datetime.timedelta(seconds=(time.time() - start_time))))
print('Pasos tomados: ', steps_taken)
print('Porcentaje de celdas limpias: ', (model.num_agents - model.count_dirty()) / model.num_agents * 100)

Tiempo de ejecución: 0:00:00.018306
Pasos tomados:  100
Porcentaje de celdas limpias:  98.0


### **Visualización**

In [5]:
all_grid = model.datacollector.get_model_vars_dataframe()

In [6]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap='Blues')

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=steps_taken)

### **Animación**

In [7]:
anim

### **Conclusiones**

En esta actividad aprendí mucho, no sólo fortalecí mi experiencia en python sino que también aprendí sobre los modelos y los agentes, y a cómo interactúan entre ellos y con su entorno. También aprendí sobre las librerías mesa, numpy y matplotlib, estas librerías resultaron muy útiles y fué muy interesante aprender a usarlas.
Todavía hay cosas que no aprendo por completo, por ejemplo hubo cosas que no entendí de get_grid y el datacollector.